In [13]:
# !pip freeze > requirements.txt
# !pip install -r requirements.txt

In [2]:
import os
import json
from datetime import datetime
import pytz
from collections import Counter
import random
random.seed(42)

import numpy as np
import torch
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from torchvision import models, transforms
from torchsummary import summary
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)

import warnings
warnings.filterwarnings("ignore")

# Preprocessing

In [3]:
class PathLabelProcessor:
    def __init__(self, 
                 base_path,
                 folder_name):
        self.base_path = base_path
        self.folder_name = folder_name
        
        self.label_images()
      
    def find_folders_by_name(self):
        matching_folders = []

        for root, dirs, files in os.walk(self.base_path):
            for dir_name in dirs:
                if self.folder_name in dir_name:
                    folder_path = os.path.join(root, dir_name)
                    matching_folders.append(folder_path)

        for folder_path in matching_folders:
            print(folder_path)
            
        return matching_folders

    def find_image_json_pairs(self):
        image_paths = []
        json_paths = []

        for folder_path in self.find_folders_by_name():
            for root, dirs, files in os.walk(folder_path):
                for image_file in filter(lambda x: x.lower().endswith(('jpg', 'png')), files):
                    image_path = os.path.join(root, image_file)
                    json_file = f"{os.path.splitext(image_path)[0]}.json"
                    if os.path.isfile(json_file):
                        image_paths.append(image_path)
                        json_paths.append(json_file)

        print(f'Total images: {len(image_paths)}, Total JSON files: {len(json_paths)}')
        
        return image_paths, json_paths

    def label_images(self):
        self.labeled_image_paths = []

        for image_path, json_path in zip(*self.find_image_json_pairs()):
            with open(json_path) as f:
                data = json.load(f)

            label = 1 if data['metaData']['lesions'] == 'A7' else 0
            self.labeled_image_paths.append((image_path, label))
        
        self.symptomatic_count = len(Counter(item for item in self.labeled_image_paths if item[1] == 0))
        self.asymptomatic_count = len(Counter(item for item in self.labeled_image_paths if item[1] == 1))
        
        print(f'Total cases: {len(self.labeled_image_paths)}')
        print(f'Number of symptomatic cases: {self.symptomatic_count}, Number of asymptomatic cases: {self.asymptomatic_count}')

In [4]:
%%time
base_path = 'skin/Train'
folder_name = '일반'

processor = PathLabelProcessor(base_path=base_path,
                               folder_name=folder_name)

labeled_image_paths = processor.labeled_image_paths
weight_class_0 = 1.0 / processor.symptomatic_count
weight_class_1 = 1.0 / processor.asymptomatic_count

skin/Train/반려견/피부/일반카메라
skin/Train/반려묘/피부/일반카메라
Total images: 433822, Total JSON files: 433822
Total cases: 433822
Number of symptomatic cases: 209523, Number of asymptomatic cases: 224299
CPU times: user 1min 14s, sys: 35.8 s, total: 1min 50s
Wall time: 10min 13s


In [ ]:
class ImageDataset(Dataset):
    def __init__(self, labeled_image_paths, transform):
        self.labeled_image_paths = labeled_image_paths
        self.transform = transform

    def __len__(self):
        return len(self.labeled_image_paths)

    def __getitem__(self, idx):
        image_path, label = self.labeled_image_paths[idx]

        image = Image.open(image_path)
        image = self.transform(image)
        
        return image, label

In [5]:
%%time
transform = transforms.Compose([transforms.Resize((240, 240)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

dataset = ImageDataset(labeled_image_paths=labeled_image_paths,
                       transform=transform)

CPU times: user 1.65 ms, sys: 931 µs, total: 2.58 ms
Wall time: 2.18 ms


In [ ]:
class DataLoaderMaker:
    def __init__(self, dataset, batch_size, train_ratio=None, num_workers=None):
        self.dataset = dataset
        self.train_ratio = train_ratio
        self.batch_size = batch_size
        self.num_workers = num_workers

        if train_ratio:
            self.split_and_make_dataloader()
        else:
            self.dataloader = self.make_dataloader(self.dataset)

    def make_dataloader(self, dataset, shuffle=False):
        dataloader = DataLoader(
            dataset=dataset,
            batch_size=self.batch_size,
            shuffle=shuffle,
            num_workers=self.num_workers,
            pin_memory=True
        )
        
        self.inspect_data(dataloader)
        
        return dataloader

    def split_and_make_dataloader(self):
        train_size = int(len(self.dataset) * self.train_ratio)
        test_size = len(self.dataset) - train_size
        train_dataset, test_dataset = random_split(self.dataset, [train_size, test_size])

        self.train_loader = self.make_dataloader(train_dataset, shuffle=True)
        self.test_loader = self.make_dataloader(test_dataset, shuffle=True)

    def inspect_data(self, dataloader):
        print("Inspecting Data...")

        class_counts = {}
        for _, labels in dataloader:
            for label in labels.tolist():
                class_counts[label] = class_counts.get(label, 0) + 1

        print("- Class Counts:")
        for class_label, count in class_counts.items():
            print(f"  Class {class_label}: {count} samples")

In [6]:
%%time
batch_size = 96
num_workers = os.cpu_count()
train_ratio = 0.8
weight_class_0 = 1.0 / processor.symptomatic_count
weight_class_1 = 1.0 / processor.asymptomatic_count
class_weights = torch.tensor([weight_class_0, weight_class_1])

data_loader = DataLoaderMaker(
    dataset=dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    train_ratio=train_ratio
)

Inspecting Data...
- Class Counts:
  Class 1: 179491 samples
  Class 0: 167566 samples
Inspecting Data...
- Class Counts:
  Class 0: 41957 samples
  Class 1: 44808 samples
CPU times: user 1min 19s, sys: 1min 23s, total: 2min 42s
Wall time: 23min 19s


# Modeling

In [7]:
class ModelTrainer:
    def __init__(self, 
                 model,
                 device,
                 train_dataloader,
                 valid_dataloader,
                 loss_fn,
                 optimizer,
                 scheduler,
                 pet_type,
                 lesion):
        self.device = device
        self.model = model.to(self.device)
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.loss_fn = loss_fn.to(device)
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.best_f1_score = 0.0
        korea = pytz.timezone('Asia/Seoul')
        now = datetime.now(korea)
        start_time = now.strftime('%Y%m%d-%H%M%S')
        self.name = f'{start_time}_{pet_type}_{lesion}.pth'
        self.writer = SummaryWriter(log_dir=f'runs/{self.name}')

    def calculate_f1_score(self, predicted, labels):
        return f1_score(
            labels,
            predicted,
            average='binary'
        )

    def calculate_auc_roc(self, predicted, labels):
        return roc_auc_score(
            labels,
            predicted
        )

    def train_one_epoch(self, epoch, num_epochs):
        self.model.train()
        total_loss = 0.0
        correct = 0
        total = 0
    
        for step, (inputs, labels) in enumerate(tqdm(self.train_dataloader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')):
            inputs, labels = inputs.to(self.device), labels.to(self.device)
    
            outputs = self.model(inputs)
            
            loss = self.loss_fn(outputs, labels)
            total_loss += loss.item()
    
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
            loss.backward()
    
            self.optimizer.step()
            self.optimizer.zero_grad()
    
        self.scheduler.step()
        self.writer.add_scalar(
            'LearningRate',
            self.scheduler.get_last_lr()[0],
            epoch
        )
    
        avg_loss = total_loss / len(self.train_dataloader)
        accuracy = correct / total
        self.writer.add_scalar('Loss/train', avg_loss, epoch)        
        self.writer.add_scalar('Accuracy/train', accuracy, epoch)

    def eval_one_epoch(self, epoch):
        self.model.eval()
        total_loss = 0.0
        correct = 0
        total = 0
        all_predicted = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in self.valid_dataloader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
        
                outputs = self.model(inputs)
        
                loss = self.loss_fn(outputs, labels)
                total_loss += loss.item()
        
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
                all_predicted.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        avg_loss = total_loss / len(self.valid_dataloader)
        accuracy = correct / total
        self.writer.add_scalar('Loss/valid', avg_loss, epoch)
        self.writer.add_scalar('Accuracy/valid', accuracy, epoch)
        
        current_f1_score = self.calculate_f1_score(
            np.array(all_predicted),
            np.array(all_labels)
        )
        current_auc_roc = self.calculate_auc_roc(
            np.array(all_predicted),
            np.array(all_labels)
        )
        
        self.writer.add_scalar('F1 Score/valid', current_f1_score, epoch)
        self.writer.add_scalar('AUC-ROC/valid', current_auc_roc, epoch)
        
        if current_f1_score > self.best_f1_score:
            self.best_f1_score = current_f1_score
            torch.save(self.model, self.name)

    def train(self, num_epochs):
        for epoch in range(num_epochs):
            self.train_one_epoch(epoch, num_epochs)
            self.eval_one_epoch(epoch)
            
        self.writer.close()

In [8]:
model = models.efficientnet_b1()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_ftrs = model.classifier[1].in_features
num_classes = 2
model.classifier[1] = nn.Linear(num_ftrs, num_classes)
loss_fn = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.AdamW(model.parameters(), weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

trainer = ModelTrainer(
    model=model,
    device=device,
    train_dataloader=data_loader.train_loader,
    valid_dataloader=data_loader.test_loader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    scheduler=scheduler
)

In [9]:
trainer.train(30)

Epoch 24/30:  33%|███▎      | 1181/3616 [10:08<20:34,  1.97batch/s]

# Evaluation

In [9]:
class ModelTester:
    def __init__(self, path, device, dataloader):
        self.device = device
        self.dataloader = dataloader
        self.load_model(path)
        self.evaluate()

    def load_model(self, path):
        self.model = torch.load(path)
        self.model.to(self.device)

    def classify(self):
        self.model.eval()
        predictions = []
        labels = []
        probabilities = []

        with torch.no_grad():
            for inputs, targets in tqdm(self.dataloader):
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                outputs = self.model(inputs)
                
                _, predicted = torch.max(outputs, 1)

                predictions.extend(predicted.cpu().numpy())
                labels.extend(targets.cpu().numpy())
                probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())

        return predictions, labels, probabilities

    def calculate_prob_stats(self, probabilities):
        probabilities = np.array(probabilities)
        min_probs = np.min(probabilities)
        max_probs = np.max(probabilities)
        std_probs = np.std(probabilities)
        mean_probs = np.mean(probabilities)

        return min_probs, max_probs, std_probs, mean_probs

    def calculate_percentage(self, value):
        return f'{value*100:.2f}%'

    def evaluate(self):
        predictions, labels, probabilities = self.classify()
        cm = confusion_matrix(labels, predictions)
        accuracy = accuracy_score(labels, predictions)
        f1 = f1_score(labels, predictions, average='weighted')

        min_probs, max_probs, std_probs, mean_probs = self.calculate_prob_stats(probabilities)

        print('Evaluation Results:')
        print(f'Confusion Matrix:\n{cm}')
        print(f'Accuracy: {self.calculate_percentage(accuracy)}')
        print(f'F1 Score: {self.calculate_percentage(f1)}')
        print(f'Mean Probability: {self.calculate_percentage(mean_probs)}')
        print(f'Max Probability: {self.calculate_percentage(max_probs)}')
        print(f'Min Probability: {self.calculate_percentage(min_probs)}')
        print(f'Standard Deviation of Probabilities: {std_probs:.4f}')

In [5]:
%%time
base_path = 'skin/Valid'
folder_name = '일반'

processor = PathLabelProcessor(base_path=base_path, folder_name=folder_name)

labeled_image_paths = processor.labeled_image_paths

skin/Valid/반려견/피부/일반카메라
skin/Valid/반려묘/피부/일반카메라
Total images: 54223, Total JSON files: 54223
Total cases: 54223
Number of symptomatic cases: 26189, Number of asymptomatic cases: 28034
CPU times: user 10.1 s, sys: 4.97 s, total: 15.1 s
Wall time: 1min 24s


In [10]:
%%time
transform = transforms.Compose([transforms.Resize((240, 240))])

dataset = ImageDataset(
    labeled_image_paths=labeled_image_paths,
    transform=transform
)

batch_size = 32
num_workers = os.cpu_count()

data_loader = DataLoaderMaker(
    dataset=dataset,
    batch_size=batch_size,
    num_workers=num_workers
)

path = '20231207-111838_symptoms.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...


- Class Counts:
  Class 0: 26189 samples
  Class 1: 28034 samples


100%|██████████| 1695/1695 [02:43<00:00, 10.40it/s]


Evaluation Results:
Confusion Matrix:
[[21207  4982]
 [ 7644 20390]]
Accuracy: 76.71%
F1 Score: 76.70%
Mean Probability: 50.00%
Max Probability: 100.00%
Min Probability: 0.00%
Standard Deviation of Probabilities: 0.3096
CPU times: user 1min 27s, sys: 20.5 s, total: 1min 48s
Wall time: 5min 22s


In [11]:
%%time
dataset = ImageDataset(
    labeled_image_paths=[item for item in labeled_image_paths if item[1] == 0],
    transform=transform)

data_loader = DataLoaderMaker(
    dataset=dataset,
    batch_size=batch_size,
    num_workers=num_workers
)

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...
- Class Counts:
  Class 0: 26189 samples


100%|██████████| 819/819 [01:18<00:00, 10.38it/s]


Evaluation Results:
Confusion Matrix:
[[21207  4982]
 [    0     0]]
Accuracy: 80.98%
F1 Score: 89.49%
Mean Probability: 50.00%
Max Probability: 99.93%
Min Probability: 0.07%
Standard Deviation of Probabilities: 0.2832
CPU times: user 41.3 s, sys: 10.8 s, total: 52.1 s
Wall time: 2min 36s


In [12]:
%%time
dataset = ImageDataset(
    labeled_image_paths=[item for item in labeled_image_paths if item[1] == 1],
    transform=transform)

data_loader = DataLoaderMaker(
    dataset=dataset,
    batch_size=batch_size,
    num_workers=num_workers
)

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...
- Class Counts:
  Class 1: 28034 samples


100%|██████████| 877/877 [01:25<00:00, 10.25it/s]


Evaluation Results:
Confusion Matrix:
[[    0     0]
 [ 7644 20390]]
Accuracy: 72.73%
F1 Score: 84.21%
Mean Probability: 50.00%
Max Probability: 100.00%
Min Probability: 0.00%
Standard Deviation of Probabilities: 0.3324
CPU times: user 44.9 s, sys: 11.3 s, total: 56.2 s
Wall time: 2min 48s


In [ ]:
class PreModelTester:
    def __init__(self, path, device, dataloader):
        self.device = device
        self.dataloader = dataloader
        self.model = models.vgg16_bn(pretrained=True)
        self.load_model(path)
        self.evaluate()

    def load_model(self, path):
        self.model = models.vgg16_bn(pretrained=True)
        nr_filters = self.model.classifier[0].in_features
        self.model.classifier = nn.Linear(nr_filters, 1)
        state_dict = torch.load(path, map_location=torch.device("cpu"))
        model_dict = self.model.state_dict()
        state_dict = {k: v for k, v in state_dict.items() if k in model_dict}
        model_dict.update(state_dict)
        self.model.load_state_dict(model_dict)
        self.model = self.model.to(self.device)

    def classify(self):
        self.model.eval()
        predictions = []
        labels = []
        probabilities = []

        with torch.no_grad():
            for inputs, targets in tqdm(self.dataloader):
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)
                outputs = self.model(inputs)
                
                probs = torch.nn.functional.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1)

                predictions.extend(predicted.cpu().numpy())
                labels.extend(targets.cpu().numpy())
                probabilities.extend(probs.max(dim=1).values.cpu().numpy())

        return predictions, labels, probabilities

    def calculate_prob_stats(self, probabilities):
        probabilities = np.array(probabilities)
        min_probs = np.min(probabilities)
        max_probs = np.max(probabilities)
        std_probs = np.std(probabilities)
        mean_probs = np.mean(probabilities)

        return min_probs, max_probs, std_probs, mean_probs

    def evaluate(self):
        predictions, labels, probabilities = self.classify()
        cm = confusion_matrix(labels, predictions)
        accuracy = accuracy_score(labels, predictions)
        f1 = f1_score(labels, predictions, average='weighted')

        min_probs, max_probs, std_probs, mean_probs = self.calculate_prob_stats(probabilities)

        print("Evaluation Results:")
        print(f"Confusion Matrix:\n{cm}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"Mean Probability: {mean_probs:.4f}")
        print(f"Max Probability: {max_probs:.4f}")
        print(f"Min Probability: {min_probs:.4f}")        
        print(f"Standard Deviation of Probabilities: {std_probs:.4f}")

In [ ]:
%%time
dataset = ImageDataset(
    labeled_image_paths=labeled_image_paths,
    transform=transform
)

data_loader = DataLoaderMaker(
    dataset=dataset,
    batch_size=batch_size,
    num_workers=num_workers)

path = 'pre_eye.pt'

PreModelTester(path=path, device=device, dataloader=data_loader.dataloader)

In [ ]:
%%time
dataset = ImageDataset(
    labeled_image_paths=labeled_image_paths,
    transform=transform
)

data_loader = DataLoaderMaker(
    dataset=dataset,
    batch_size=batch_size,
    num_workers=num_workers)

PreModelTester(path=path, device=device, dataloader=data_loader.dataloader)

In [ ]:
%%time
dataset = ImageDataset(
    labeled_image_paths=labeled_image_paths,
    transform=transform
)

data_loader = DataLoaderMaker(
    dataset=dataset,
    batch_size=batch_size,
    num_workers=num_workers)

PreModelTester(path=path, device=device, dataloader=data_loader.dataloader)